In [22]:
import sys
sys.path.append("D:/ML_Study/2017-CCF-BDCI-AIJudge")

import numpy as np
import pandas as pd
from gensim.models import Doc2Vec
from utils import LOGGER
from config.db_config import Config
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score

import warnings
warnings.filterwarnings('ignore')
config = Config()

In [3]:
import sklearn
sklearn.__version__

'0.19.1'

## 加载标签数据

In [110]:
data_df = pd.read_csv(config.data_csv_path, encoding='utf-8', sep=',')
data_df = data_df.dropna()
data_df['penalty'] = data_df['penalty'].astype(int)
y = data_df['penalty'] - 1
rows = data_df.shape[0]
labels = np.unique(y)
num_class = len(labels)
print(rows,num_class,labels)
data_df.head()

100 8 [0 1 2 3 4 5 6 7]


,id,content,laws,penalty
0,16,公诉 机关 霍邱县 人民检察院 被告人 许某 1975 日生 2012 因涉嫌 危险 驾驶 ...,"133,72,73",3
1,32,公诉 机关 海口市 龙华区 人民检察院 被告人 王某 海口市 龙华区 人民检察院 海龙 检公...,"347,67,52,64",1
2,41,公诉 机关 广东省 潮州市 人民检察院 被告人 覃学彬 1980 出生 广西壮族自治区 大新...,"263,25,52,53,55,56",5
3,57,公诉 机关 榆林市 榆阳区 人民检察院 上诉人 原审 被告人 2012 因涉嫌 盗窃罪 榆林...,"264,52,53,67,72,73",5
4,60,公诉 机关 榆阳区 人民检察院 上诉人 原审 被告人 刘某 汉族 陕西省 横山县 小学文化 ...,"224,25,26,27,52,72",7


In [111]:
data_df.isnull().any()

id         False
content    False
laws       False
penalty    False
dtype: bool

# 加载pre-train模型

In [139]:
# 模型加载-初始化文档的特征
print("2. load doc2vec model.init word vectors......")
pre_train_model = Doc2Vec.load(config.model_d2v_dm)
X = np.array([pre_train_model.docvecs[idx] for idx in range(rows)])
print("dataset size rows={0},cols={1}".format(X.shape[0], X.shape[1]))

2. load doc2vec model.init word vectors......
dataset size rows=100,cols=300


## 切分数据集/初始化数据矩阵向量

In [146]:
# 切分数据： 训练集，测试集
print("3. split dataset train and test,init stack matrix ......")
TR = int(0.7*rows)
X_train,X_test = X[:TR],X[TR:]
y_train, y_test = y[0:TR],y[TR:]
print("dataset size X_train={0},X_test={1},dim={2}".format(X_train.shape[0], X_test.shape[0], X_test.shape[1]))
print(np.unique(y_train),np.unique(y_test))
# 初始化数据矩阵，默认数据为0
stack = np.zeros((X_train.shape[0], num_class))
stack_te = np.zeros((X_test.shape[0], num_class))
print(stack.shape,stack_te.shape)

3. split dataset train and test,init stack matrix ......
dataset size X_train=70,X_test=30,dim=300
[0 1 2 3 4 5 6 7] [0 1 2 3 4 5 6]
(70, 8) (30, 8)


In [147]:
 print(len(y_train),len(X_train))

70 70


In [148]:
skf = StratifiedKFold(n_splits=8, random_state=config.seed)
for i, (train_index, val_index) in enumerate(skf.split(X_train, y_train)):
    

[ 3 12 13 14 15 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35
 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59
 60 61 62 63 64 65 66 67 68 69] 3     4
12    1
13    7
14    4
15    1
17    0
18    6
19    6
20    6
21    1
22    6
23    2
24    3
25    1
26    1
27    6
28    7
29    1
30    2
31    6
32    0
33    6
34    1
35    3
36    1
37    6
38    0
39    0
40    1
41    1
42    6
43    2
44    6
45    3
46    1
47    2
48    5
49    6
50    0
51    1
52    6
53    4
54    1
55    6
56    1
57    1
58    0
59    0
60    1
61    1
62    3
63    5
64    1
65    4
66    1
67    0
68    5
69    2
Name: penalty, dtype: int32
[ 0  1  2  4  5  6  7  8  9 10 11 14 16 20 22 25 26 27 28 29 30 31 33 34
 35 36 37 38 39 40 41 42 43 44 45 46 47 49 50 51 52 53 54 55 56 57 58 59
 60 61 62 63 64 65 66 67 68 69] 0     2
1     0
2     4
4     6
5     0
6     6
7     1
8     1
9     1
10    3
11    7
14    4
16    5
20    6
22    6
25    1
26    1
27    6
28    7


In [90]:
print(len(X_test),len(X_test))

30 30


## 交叉验证- 特征得分平均

In [91]:
k = 8
print("cross_validate k={0}".format(k))
score_va = 0
score_te = 0
skf = StratifiedKFold(n_splits=k, random_state=config.seed)
for i, (train_index, val_index) in enumerate(skf.split(X_train, y_train)):
    print("cross_validate.....i={0}".format(i + 1))
    train_data, val_data = X_train[train_index], X_train[val_index]
    train_y, val_y = y_train[train_index], y_train[val_index]
    print(train_index)
    print(train_y)
    print(train_data)

cross_validate k=8
cross_validate.....i=1
[ 9 11 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35
 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59
 60 61 62 63 64 65 66 67 68 69]
9     NaN
11    NaN
14    5.0
15    2.0
16    NaN
17    1.0
18    NaN
19    7.0
20    7.0
21    NaN
22    7.0
23    NaN
24    4.0
25    2.0
26    2.0
27    NaN
28    NaN
29    2.0
30    3.0
31    7.0
32    1.0
33    NaN
34    NaN
35    4.0
36    2.0
37    7.0
38    1.0
39    NaN
40    2.0
41    NaN
42    NaN
43    NaN
44    7.0
45    4.0
46    2.0
47    3.0
48    6.0
49    7.0
50    1.0
51    2.0
52    7.0
53    NaN
54    2.0
55    NaN
56    NaN
57    NaN
58    NaN
59    NaN
60    2.0
61    2.0
62    NaN
63    6.0
64    2.0
65    5.0
66    2.0
67    1.0
68    NaN
69    3.0
Name: penalty, dtype: float64
[[-1.2607483e-02 -5.4897770e-04  3.7923502e-03 ...  3.2193456e-03
  -5.1823976e-03 -1.4352198e-02]
 [-3.5662673e-02  1.9878896e-04  1.3963391e-02 ...  1.4691130e-02
  -1.